In [ ]:
# Install required packages for SD 2.1 and Diffusers with LoRA support
!pip install bitsandbytes transformers accelerate peft datasets -q
!pip install git+https://github.com/huggingface/diffusers.git -q
# Download training script for SD 2.1 (with LoRA support)
!wget https://raw.githubusercontent.com/huggingface/diffusers/refs/heads/main/examples/text_to_image/train_text_to_image_lora.py -O train_text_to_image_lora.py

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
--2025-04-24 19:09:51--  https://raw.githubusercontent.com/huggingface/diffusers/refs/heads/main/examples/text_to_image/train_text_to_image_lora.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40568 (40K) [text/plain]
Saving to: ‘train_text_to_image_lora.py’

train_text_to_image 100%[===================>]  39.62K  --.-KB/s    in 0.001s  

2025-04-24 19:09:51 (34.9 MB/s) - ‘train_text_to_image_lora.py’ saved [40568/40568]



In [ ]:
!pip install xformers

In [ ]:
from huggingface_hub import login
from google.colab import userdata

notebook_key=userdata.get('notebooks')
login(notebook_key)

In [ ]:
!python train_text_to_image_lora.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1-base" \
  --dataset_name="tomytjandra/h-and-m-fashion-caption-12k" \
  --caption_column="text" \
  --resolution=512 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=8 \
  --lr_scheduler="cosine_with_restarts" \
  --lr_warmup_steps=500 \
  --learning_rate=2e-4 \
  --max_train_steps=6000 \
  --checkpointing_steps=500 \
  --max_train_samples=10000 \
  --validation_prompt="a red dress on plain white background, product photo" \
  --validation_epochs=2 \
  --seed=42 \
  --output_dir="sd21-hm-fashion-lora" \
  --num_train_epochs=100 \
  --enable_xformers_memory_efficient_attention \
  --rank=16 \
  --allow_tf32

2025-04-24 19:10:00.608244: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 19:10:00.625743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745521800.646742   10120 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745521800.653163   10120 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 19:10:00.674592: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers.loaders import AttnProcsLayers
from diffusers.models.attention_processor import LoRAAttnProcessor

def inference_lora_sd(
    prompt: str,
    lora_model_path: str,
    negative_prompt: str = "low quality, bad quality, blurry",
    num_images: int = 1,
    num_inference_steps: int = 30,
    guidance_scale: float = 7.5,
    seed: int = None
):
    """
    Generate images using a LoRA fine-tuned Stable Diffusion 2.1 model.
    Automatically adds "on plain white background" to every prompt.

    Args:
        prompt: Text prompt for image generation (will be enhanced)
        lora_model_path: Path to the fine-tuned LoRA weights file
        negative_prompt: Negative prompt to guide generation away from undesired results
        num_images: Number of images to generate
        num_inference_steps: Number of denoising steps (more = higher quality, slower)
        guidance_scale: How closely to follow the prompt (higher = more faithful but less diverse)
        seed: Random seed for reproducibility

    Returns:
        List of generated images
    """
    # Enhance prompt with plain white background specification
    enhanced_prompt = f"{prompt}, on plain white background, isolated product photo"

    # Set random seed if provided
    if seed is not None:
        torch.manual_seed(seed)

    # Load base model
    pipeline = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        torch_dtype=torch.float16
    ).to("cuda")

    # Load LoRA weights
    pipeline.unet.load_attn_procs(lora_model_path)

    # Generate images
    images = pipeline(
        prompt=enhanced_prompt,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_images,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale
    ).images

    print(f"Generated with prompt: '{enhanced_prompt}'")
    return images

# Example usage
if __name__ == "__main__":
    image = inference_lora_sd(
        prompt="simple purple check tshirt",
        lora_model_path="./sd21-hm-fashion-lora",  # Path to your trained LoRA weights
        seed=42
    )

    image[0].save(f"fashion_item.png")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/diffusers/loaders/unet.py:212: FutureWarning: `load_attn_procs` is deprecated and will be removed in version 0.40.0. Using the `load_attn_procs()` method has been deprecated and will be removed in a future version. Please use `load_lora_adapter()`.
  deprecate("load_attn_procs", "0.40.0", deprecation_message)


  0%|          | 0/30 [00:00<?, ?it/s]

Generated with prompt: 'simple purple check tshirt, on plain white background, isolated product photo'


In [ ]:
import shutil

shutil.make_archive('/content/sd21-hm-fashion-lora', 'zip', '/content/sd21-hm-fashion-lora')

'/content/sd21-hm-fashion-lora.zip'

In [ ]:
from google.colab import files

files.download('/content/sd21-hm-fashion-lora.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>